In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import re
import math

In [2]:
df = pd.read_json("rawdatajson/products.json", lines=True)

In [3]:
df.head()

,_id,_keywords,additives,additives_debug_tags,additives_n,additives_old_n,additives_old_tags,additives_original_tags,additives_prev,additives_prev_n,...,stores_debug_tags,stores_tags,traces,traces_debug_tags,traces_hierarchy,traces_tags,unique_scans_n,unknown_nutrients_tags,update_key,with_sweeteners
0,96085745,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,[franprix],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50097852,"[anti-trace, hygiene, alcool, bille, motionsen...",[ aqua -> fr:aqua ] [ aluminium-zirconium-p...,[],7,2,"[en:e173, en:e460]",NaN,NaN,NaN,...,[],[franprix],,[],[],[],NaN,[],NaN,NaN
2,3560070272426,"[deodorant, musk, stick, carrefour, hygiene, a...",[ aqua -> fr:aqua ] [ dipropylene-glycol ->...,"[en-e470-fr-added, en-e490-fr-added, en-e1520-...",3,2,"[en:e1520, en:e110]",NaN,NaN,NaN,...,NaN,[carrefour-city],,NaN,[],[],NaN,[],NaN,NaN
3,3560070937493,"[lait, le, tolerance, sou, corp, dermatologiqu...",[ aqua -> fr:aqua ] [ paraffinum-liquidum -...,"[en-e422-fr-added, en-e211-fr-added, en-e524-f...",{u'$numberLong': u'5'},{u'$numberLong': u'2'},"[en:e307a, en:e330]",NaN,NaN,NaN,...,NaN,[carrefour],,NaN,[],[],NaN,[],NaN,NaN
4,3560071001308,"[karite, beurre, et, de, musquee, huile, carre...",NaN,[],NaN,NaN,[],NaN,NaN,NaN,...,[],[carrefour],,[],[],[],NaN,[],NaN,NaN


In [4]:
list(df.columns)

[u'_id',
 u'_keywords',
 u'additives',
 u'additives_debug_tags',
 u'additives_n',
 u'additives_old_n',
 u'additives_old_tags',
 u'additives_original_tags',
 u'additives_prev',
 u'additives_prev_n',
 u'additives_prev_original_tags',
 u'additives_prev_tags',
 u'additives_tags',
 u'additives_tags_n',
 u'allergens',
 u'allergens_hierarchy',
 u'allergens_tags',
 u'brands',
 u'brands_debug_tags',
 u'brands_tags',
 u'categories',
 u'categories_debug_tags',
 u'categories_hierarchy',
 u'categories_prev_hierarchy',
 u'categories_prev_tags',
 u'categories_tags',
 u'checkers_tags',
 u'cities_tags',
 u'code',
 u'codes_tags',
 u'complete',
 u'completed_t',
 u'correctors_tags',
 u'countries',
 u'countries_beforescanbot',
 u'countries_debug_tags',
 u'countries_hierarchy',
 u'countries_tags',
 u'created_t',
 u'creator',
 u'debug_param_sorted_langs',
 u'debug_tags',
 u'editors',
 u'editors_tags',
 u'emb_codes',
 u'emb_codes_20141016',
 u'emb_codes_debug_tags',
 u'emb_codes_orig',
 u'emb_codes_tags',
 u'

In [5]:
df.categories.value_counts()

                                                                                                                           1021
Cheveux,Shampoings                                                                                                          156
Hygiène,Gels douche,Douches et bains                                                                                        111
Cheveux,Shampoings,Shampoings anti-pelliculaires                                                                             65
Non alimentaire,Open Beauty Facts                                                                                            59
Hygiène,Savons                                                                                                               57
Hygiène,Douches et bains,Crèmes douche                                                                                       47
Hygiène,Dentifrices                                                                                     

In [26]:
df.shape 

(4346, 326)

In [6]:
df[1500:1520]

,_id,_keywords,additives,additives_debug_tags,additives_n,additives_old_n,additives_old_tags,additives_original_tags,additives_prev,additives_prev_n,...,stores_debug_tags,stores_tags,traces,traces_debug_tags,traces_hierarchy,traces_tags,unique_scans_n,unknown_nutrients_tags,update_key,with_sweeteners
1500,0738678240472,"[ultra, gliding, gel, rasage, huile, shave, cr...",[ olive-oil-peg-7-esters -> fr:olive-oil-peg-...,[],{u'$numberLong': u'1'},{u'$numberLong': u'1'},[en:e321],NaN,NaN,NaN,...,NaN,[the-barber-lodge],,NaN,[],[],NaN,[],NaN,NaN
1501,0738678239940,"[shaving, gel, crew]",[ aqua -> en:aqua ] [ dicaprylyl-carbonate ...,"[en-e502-other-added, en-e422-other-added, en-...",{u'$numberLong': u'4'},{u'$numberLong': u'0'},[],NaN,NaN,NaN,...,NaN,[],,NaN,[],[],NaN,[],NaN,NaN
1502,20553531,[cien],NaN,[],NaN,NaN,[],NaN,NaN,NaN,...,NaN,[lidl],,NaN,[],[],NaN,[],NaN,NaN
1503,06582,"[san, composant, du, bille, packaging, pa, deo...",[ aqua -> fr:aqua ] [ aluminum-chlorohydrat...,"[en-e173-fr-added, en-e211-fr-added, en-e490-f...",{u'$numberLong': u'3'},{u'$numberLong': u'1'},[en:e1520],NaN,NaN,NaN,...,NaN,[yves-rocher],,NaN,[],[],NaN,[],NaN,NaN
1504,3538394920033,"[cosly, normaux, pre, cosmetique, la, shampoin...",[ aqua -> fr:aqua ] [ eau -> fr:eau ] [ s...,"[en-e470-fr-added, en-e508-fr-added, en-e422-f...",{u'$numberLong': u'7'},{u'$numberLong': u'0'},[],NaN,NaN,NaN,...,NaN,[biocoop-la-marigoule],,NaN,[],[],NaN,[],NaN,NaN
1505,3538394612938,"[cosly, douche, savon, san, ultra, doux, fruit...",[ aqua -> fr:aqua ] [ water -> fr:water ] ...,"[en-e470-fr-added, en-e422-fr-added, en-e508-f...",12,3,"[en:e640, en:e307a, en:e330]",NaN,NaN,NaN,...,NaN,[l-eau-vive],,NaN,[],[],NaN,[],NaN,NaN
1506,3538394612839,"[cosly, savon, san, synthese, ulta, aux, cosme...",[ aqua -> fr:aqua ] [ water -> fr:water ] ...,"[en-e470-fr-added, en-e422-fr-added, en-e508-f...",{u'$numberLong': u'11'},{u'$numberLong': u'3'},"[en:e640, en:e307a, en:e330]",NaN,NaN,NaN,...,NaN,[l-eau-vive],,NaN,[],[],NaN,[],NaN,NaN
1507,3538394612730,"[cosly, douche, savon, san, ultra, doux, aux, ...",[ aqua -> fr:aqua ] [ water -> fr:water ] ...,"[en-e470-fr-added, en-e508-fr-added, en-e422-f...",9,2,"[en:e640, en:e307a]",NaN,NaN,NaN,...,NaN,[l-eau-vive],,NaN,[],[],NaN,[],NaN,NaN
1508,3532281624608,"[comestible, quality, fraise, de, pour, ph, ge...",[ aqua -> fr:aqua ] [ glycerin -> en:e422 ...,[],5,0,[],NaN,NaN,NaN,...,[],[carrefour],,[],[],[],NaN,[],NaN,NaN
1509,3532281625605,"[san, compatible, ph, le, neutre, body, la, ge...",[ glycerin -> en:e422 -> exists ] [ polyso...,"[en-e422-fr-added, en-e270-fr-added]",{u'$numberLong': u'3'},{u'$numberLong': u'1'},[en:e432],NaN,NaN,NaN,...,NaN,[auchan],,NaN,[],[],NaN,[],NaN,NaN


In [7]:
allergens = [x for x in list(df.allergens_tags.values) if type(x) == list and len(x)>1]

In [8]:
allergens

[[u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'aluminum', u'', u'', u'cinnamal', u'', u'', u'citral'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'macadamia'],
 [u'arachidyl', u'', u'', u'arachidyl'],
 [u'prunus-amygdalus', u'', u'', u'sweet-almond'],
 [u'macadamia', u'', u'', u'prunus-amygdalus'],
 [u'd-limonene', u'', u'', u'linalool', u'', u'', u'geraniol'],
 [u'linalool',
  u'',
  u'',
  u'geraniol',
  u'',
  u'',
  u'alpha-isomethyl-ionone',
  u'',
  u'',
  u'coumarin',
  u'',
  u'',
  u'citronellol']]

In [9]:
### what are these allergens?
### how many allergens are there?
len(allergens)

15

In [10]:
####
## prunus-amygdalus is almond, a tree nut
## aluminum is aluminum
## cinnamal is the naturally occuring chemical in cinnamon that gives it its flavor, banned for cosmetic use in EU
## citral is a chemical commonly found in lemon scented plants that is known to be a skin irritant
## macadamia is macademia nut, a tree nut
## arachidyl is derived from peanuts
## d-limonene is a constitutent of citurs and is known to cause skin irritation in some individuals
## linalool is a naturally occuring scent in lavendar, mint, orange blossoms, cardamom and others.  It causes skin allergies in about 5% of the Swedish population
## geraniol is naturally occuring and found in many essential oils that smell like roses.  It is banned in EU as a skin irritant
## alpha-isomethyl-ionone is an ingredient in bubble bath and aftershave that can cause skin irritation
## coumarin gives many plants their sweet smell and is a known human irritant.  Banned in EU
## citronellol is found in roses and geraniums.  It is restricted in perfume

### Sources https://www.ewg.org/skindeep/ , http://en.wikipedia.org

In [11]:
main_allergens = [x for x in list(df.allergens.values) if type(x) == unicode and len(x)>1]

In [12]:
type(df.allergens[75])

unicode

In [13]:
main_allergens

[u'MACADAMIA',
 u'PRUNUS AMYGDALUS',
 u'prunus amygdalus',
 u'PRUNUS AMYGDALUS, ALMOND',
 u'PRUNUS AMYGDALUS',
 u'Prunus Amygdalus, Prunus Amygdalus',
 u'Prunus Amygdalus',
 u'Sweet Almond',
 u'prunus amygdalus',
 u'prunus amygdalus',
 u'ALMOND',
 u'ALMOND',
 u'PRUNUS AMYGDALUS',
 u'Prunus Amygdalus',
 u'prunus amygdalus, almond',
 u'PRUNUS AMYGDALUS, ALMOND',
 u'MACADAMIA',
 u'MACADAMIA',
 u'PRUNUS AMYGDALUS, ALMOND',
 u'PRUNUS AMYGDALUS, ALMOND',
 u' aluminum, cinnamal, citral',
 u'sodium monofluorophosphate',
 u'prunus amygdalus',
 u'prunus amygdalus',
 u'PRUNUS AMYGDALUS, ALMOND',
 u'PRUNUS AMYGDALUS, ALMOND',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus, Macadamia',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus',
 u'prunus amygdalus',
 u'arachidyl, arachidyl',
 u'Corylus avellana',
 u'wheat',
 u'prunus amygdalus',
 u'PRUNUS AMYGDALUS',
 u'Prunus Amygdalus',
 u'Prunus Amygdalus, Sweet Almond',
 u'MACADAMIA, PRUNUS AMYGDALUS

In [14]:
len(main_allergens)

58

In [40]:
cats = [x for x in df['categories_hierarchy'] if type(x) == list  and len(x) >0]

In [41]:
type(df['categories_hierarchy'][0])

float

In [16]:
len(cats)

2619

In [17]:
### half of all cosmetics have category hierarchies

In [18]:
cats_general = [x for x in df['categories'] if type(x) == unicode and len(x)>0]

In [19]:
len(cats_general)

2619

In [20]:
cats_general

[u'Hygi\xe8ne,D\xe9odorants anti-transpirants,D\xe9odorants,D\xe9odorants bille',
 u'Hygi\xe8ne,D\xe9odorants,D\xe9odorants sticks',
 u'Laits pour le corps',
 u'\xc9pilation',
 u'Hygi\xe8ne,Hygi\xe8ne intime,Gels pour la toilette intime',
 u'Cr\xe8mes pour les pieds',
 u'Cr\xe8mes pour les pieds',
 u'Hygi\xe8ne,Savons,Savons liquides',
 u'Savons liquides',
 u'Baumes apr\xe8s-rasage',
 u'Cheveux,Shampoings,Shampoings anti-pelliculaires',
 u'Cheveux,Shampoings,Shampoings anti-pelliculaires',
 u'Soins cheveux',
 u'Savons liquides',
 u'Savons liquides',
 u'Cheveux,Hygi\xe8ne,Shampoings,Shampoings 2-en-1,Shampoings + Gels douche,Gels douche,Douches et bains',
 u'Accessoires,Lingettes impr\xe9gn\xe9es,Lingettes pour b\xe9b\xe9',
 u'Hygi\xe8ne,Gels douche,Douches et bains',
 u'Cheveux,Shampoings,Shampoings anti-pelliculaires',
 u'Cheveux,Shampoings,Shampoings pour cheveux secs',
 u'Cheveux,Apr\xe8s-shampoings',
 u'Cheveux,Shampoings',
 u'Cheveux,Apr\xe8s-shampoings,Apr\xe8s-shampoings pour ch

In [21]:
cat_tags = [x for x in df['categories_tags'] if type(x) == list  and len(x) >0]

In [22]:
cat_tags

[[u'en:hygiene',
  u'en:anti-perspirants',
  u'en:deodorants',
  u'en:roll-on-deodorants'],
 [u'en:hygiene', u'en:deodorants', u'fr:deodorants-sticks'],
 [u'en:body', u'en:body-milks'],
 [u'en:hygiene', u'en:hair-removal'],
 [u'en:hygiene', u'en:intimate-hygiene', u'fr:gels-pour-la-toilette-intime'],
 [u'en:body', u'en:body-creams', u'fr:cremes-pour-les-pieds'],
 [u'en:body', u'en:body-creams', u'fr:cremes-pour-les-pieds'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:shaving', u'en:aftershaves', u'fr:baumes-apres-rasage'],
 [u'en:hair', u'en:shampoos', u'en:anti-dandruff-shampoos'],
 [u'en:hair', u'en:shampoos', u'en:anti-dandruff-shampoos'],
 [u'en:hair', u'en:hair-care'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hair',
  u'en:hygiene',
  u'en:shampoos',
  u'en:2-in-1-shampoos',
  u'en:shampoos-shower-gels',
  u'en:shower-gels',
  u'en:shower

In [23]:
len(cat_tags)

2619

In [25]:
for x in df.columns:
    print x, type(df[x][0])

_id <type 'unicode'>
_keywords <type 'list'>
additives <type 'float'>
additives_debug_tags <type 'float'>
additives_n <type 'float'>
additives_old_n <type 'float'>
additives_old_tags <type 'float'>
additives_original_tags <type 'float'>
additives_prev <type 'float'>
additives_prev_n <type 'numpy.float64'>
additives_prev_original_tags <type 'float'>
additives_prev_tags <type 'float'>
additives_tags <type 'float'>
additives_tags_n <type 'numpy.float64'>
allergens <type 'float'>
allergens_hierarchy <type 'float'>
allergens_tags <type 'float'>
brands <type 'float'>
brands_debug_tags <type 'float'>
brands_tags <type 'float'>
categories <type 'float'>
categories_debug_tags <type 'float'>
categories_hierarchy <type 'float'>
categories_prev_hierarchy <type 'float'>
categories_prev_tags <type 'float'>
categories_tags <type 'float'>
checkers_tags <type 'list'>
cities_tags <type 'float'>
code <type 'unicode'>
codes_tags <type 'list'>
complete <type 'dict'>
completed_t <type 'float'>
correctors_ta

In [59]:
### check for completeness
cat_completeness = []

for x in df.columns:
    a = x
    t = type(df[x][0])
    ln = df[x].notnull().sum()
    threshold = ['Over Threshold' if ln > 2619 else 'Under Threshold'][0]
    cat_completeness.append({'cat_name': a, 
                            'cat_type': t,
                            'cat_valid_count': ln,
                            'cat_completeness': threshold})

In [60]:
cat_completeness[:20]

[{'cat_completeness': 'Over Threshold',
  'cat_name': u'_id',
  'cat_type': unicode,
  'cat_valid_count': 4346},
 {'cat_completeness': 'Over Threshold',
  'cat_name': u'_keywords',
  'cat_type': list,
  'cat_valid_count': 4346},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives',
  'cat_type': float,
  'cat_valid_count': 2291},
 {'cat_completeness': 'Over Threshold',
  'cat_name': u'additives_debug_tags',
  'cat_type': float,
  'cat_valid_count': 4148},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_n',
  'cat_type': float,
  'cat_valid_count': 2296},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_old_n',
  'cat_type': float,
  'cat_valid_count': 2296},
 {'cat_completeness': 'Over Threshold',
  'cat_name': u'additives_old_tags',
  'cat_type': float,
  'cat_valid_count': 4148},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_original_tags',
  'cat_type': float,
  'cat_valid_count': 789},
 {'cat_completeness': 'U

In [64]:
### how many were not complete?
cols2 = [x['cat_name'] for x in cat_completeness if x['cat_completeness'] == 'Over Threshold']
len(cols2)

100

In [67]:
### create a df that only contains the completed columns
df2 = df[cols2]

In [68]:
df2.head()

,_id,_keywords,additives_debug_tags,additives_old_tags,additives_tags,allergens,allergens_hierarchy,allergens_tags,brands,brands_tags,...,sortkey,states,states_hierarchy,states_tags,stores,stores_tags,traces,traces_hierarchy,traces_tags,unknown_nutrients_tags
0,96085745,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{u'$numberLong': u'464871799'},"en:to-be-completed, en:nutrition-facts-to-be-c...","[en:to-be-completed, en:nutrition-facts-to-be-...","[en:to-be-completed, en:nutrition-facts-to-be-...",Franprix,[franprix],NaN,NaN,NaN,NaN
1,50097852,"[anti-trace, hygiene, alcool, bille, motionsen...",[],"[en:e173, en:e460]","[en:e173, en:e422, en:e428, en:e466, en:e211, ...",,[],[],"Rexona,Unilever","[rexona, unilever]",...,1487927561,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Franprix,[franprix],,[],[],[]
2,3560070272426,"[deodorant, musk, stick, carrefour, hygiene, a...","[en-e470-fr-added, en-e490-fr-added, en-e1520-...","[en:e1520, en:e110]","[en:e470, en:e490, en:e110]",,[],[],Carrefour,[carrefour],...,1491076400,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour City,[carrefour-city],,[],[],[]
3,3560070937493,"[lait, le, tolerance, sou, corp, dermatologiqu...","[en-e422-fr-added, en-e211-fr-added, en-e524-f...","[en:e307a, en:e330]","[en:e422, en:e211, en:e524, en:e307a, en:e330]",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459971277'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
4,3560071001308,"[karite, beurre, et, de, musquee, huile, carre...",[],[],[],,[],[],Carrefour,[carrefour],...,487925728,"en:to-be-completed, en:nutrition-facts-complet...","[en:to-be-completed, en:nutrition-facts-comple...","[en:to-be-completed, en:nutrition-facts-comple...",Carrefour,[carrefour],,[],[],[]


In [72]:
df2.ix[3]

_id                                                                    3560070937493
_keywords                          [lait, le, tolerance, sou, corp, dermatologiqu...
additives_debug_tags               [en-e422-fr-added, en-e211-fr-added, en-e524-f...
additives_old_tags                                               [en:e307a, en:e330]
additives_tags                        [en:e422, en:e211, en:e524, en:e307a, en:e330]
allergens                                                                           
allergens_hierarchy                                                               []
allergens_tags                                                                    []
brands                                                                     Carrefour
brands_tags                                                              [carrefour]
categories                                                       Laits pour le corps
categories_debug_tags                                            

In [73]:
df2.shape

(4346, 100)

In [96]:
### drop rows that are not relevant.  Define relevant as if categories_tags is filled
idx = []
for i, val in enumerate(df2['categories_tags']):
    #print i, val, type(val)
    if (type(val) == float) or (type(val) == list and len(val)  <1):
        idx.append(i)

In [97]:
len(idx)

1727

In [98]:
idx[:5]

[0, 5, 20, 39, 55]

In [99]:
df2['categories_tags'].head(21)

0                                                   NaN
1     [en:hygiene, en:anti-perspirants, en:deodorant...
2     [en:hygiene, en:deodorants, fr:deodorants-sticks]
3                              [en:body, en:body-milks]
4                         [en:hygiene, en:hair-removal]
5                                                    []
6     [en:hygiene, en:intimate-hygiene, fr:gels-pour...
7     [en:body, en:body-creams, fr:cremes-pour-les-p...
8     [en:body, en:body-creams, fr:cremes-pour-les-p...
9            [en:hygiene, en:soaps, fr:savons-liquides]
10           [en:hygiene, en:soaps, fr:savons-liquides]
11    [en:shaving, en:aftershaves, fr:baumes-apres-r...
12    [en:hair, en:shampoos, en:anti-dandruff-shampoos]
13    [en:hair, en:shampoos, en:anti-dandruff-shampoos]
14                              [en:hair, en:hair-care]
15           [en:hygiene, en:soaps, fr:savons-liquides]
16           [en:hygiene, en:soaps, fr:savons-liquides]
17    [en:hair, en:hygiene, en:shampoos, en:2-in

In [100]:
df3 = df2.drop(df2.index[idx])

In [101]:
df3.shape

(2619, 100)

In [102]:
df3.head()

,_id,_keywords,additives_debug_tags,additives_old_tags,additives_tags,allergens,allergens_hierarchy,allergens_tags,brands,brands_tags,...,sortkey,states,states_hierarchy,states_tags,stores,stores_tags,traces,traces_hierarchy,traces_tags,unknown_nutrients_tags
1,50097852,"[anti-trace, hygiene, alcool, bille, motionsen...",[],"[en:e173, en:e460]","[en:e173, en:e422, en:e428, en:e466, en:e211, ...",,[],[],"Rexona,Unilever","[rexona, unilever]",...,1487927561,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Franprix,[franprix],,[],[],[]
2,3560070272426,"[deodorant, musk, stick, carrefour, hygiene, a...","[en-e470-fr-added, en-e490-fr-added, en-e1520-...","[en:e1520, en:e110]","[en:e470, en:e490, en:e110]",,[],[],Carrefour,[carrefour],...,1491076400,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour City,[carrefour-city],,[],[],[]
3,3560070937493,"[lait, le, tolerance, sou, corp, dermatologiqu...","[en-e422-fr-added, en-e211-fr-added, en-e524-f...","[en:e307a, en:e330]","[en:e422, en:e211, en:e524, en:e307a, en:e330]",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459971277'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
4,3560071001308,"[karite, beurre, et, de, musquee, huile, carre...",[],[],[],,[],[],Carrefour,[carrefour],...,487925728,"en:to-be-completed, en:nutrition-facts-complet...","[en:to-be-completed, en:nutrition-facts-comple...","[en:to-be-completed, en:nutrition-facts-comple...",Carrefour,[carrefour],,[],[],[]
6,3560070416974,"[la, intime, carrefour, bio, gel, ecocert, pou...","[en-e422-fr-added, en-e470-fr-added, en-e1519-...","[en:e415, en:e330, en:e307a]","[en:e422, en:e470, en:e415, en:e1519, en:e211,...",,[],[],"Carrefour Bio,Carrefour","[carrefour-bio, carrefour]",...,{u'$numberLong': u'1463774184'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]


In [111]:
df4 = df3.reindex(index =[x for x in range(df3.shape[0])] )

In [113]:
df4.head(10)

,_id,_keywords,additives_debug_tags,additives_old_tags,additives_tags,allergens,allergens_hierarchy,allergens_tags,brands,brands_tags,...,sortkey,states,states_hierarchy,states_tags,stores,stores_tags,traces,traces_hierarchy,traces_tags,unknown_nutrients_tags
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50097852,"[anti-trace, hygiene, alcool, bille, motionsen...",[],"[en:e173, en:e460]","[en:e173, en:e422, en:e428, en:e466, en:e211, ...",,[],[],"Rexona,Unilever","[rexona, unilever]",...,1487927561,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Franprix,[franprix],,[],[],[]
2,3560070272426,"[deodorant, musk, stick, carrefour, hygiene, a...","[en-e470-fr-added, en-e490-fr-added, en-e1520-...","[en:e1520, en:e110]","[en:e470, en:e490, en:e110]",,[],[],Carrefour,[carrefour],...,1491076400,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour City,[carrefour-city],,[],[],[]
3,3560070937493,"[lait, le, tolerance, sou, corp, dermatologiqu...","[en-e422-fr-added, en-e211-fr-added, en-e524-f...","[en:e307a, en:e330]","[en:e422, en:e211, en:e524, en:e307a, en:e330]",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459971277'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
4,3560071001308,"[karite, beurre, et, de, musquee, huile, carre...",[],[],[],,[],[],Carrefour,[carrefour],...,487925728,"en:to-be-completed, en:nutrition-facts-complet...","[en:to-be-completed, en:nutrition-facts-comple...","[en:to-be-completed, en:nutrition-facts-comple...",Carrefour,[carrefour],,[],[],[]
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3560070416974,"[la, intime, carrefour, bio, gel, ecocert, pou...","[en-e422-fr-added, en-e470-fr-added, en-e1519-...","[en:e415, en:e330, en:e307a]","[en:e422, en:e470, en:e415, en:e1519, en:e211,...",,[],[],"Carrefour Bio,Carrefour","[carrefour-bio, carrefour]",...,{u'$numberLong': u'1463774184'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
7,3560070231584,"[le, gommant, corne, pied, creme, carrefour, p...",[en-e218-fr-added],"[en:e900, en:e415, en:e330, en:e307a, en:e321]","[en:e900, en:e218, en:e415, en:e330, en:e307a,...",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459949015'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
8,3560070231614,"[le, nourrissant, pied, creme, carrefour, pour...","[en-e927b-fr-added, en-e570-fr-added, en-e490-...","[en:e1520, en:e900, en:e330, en:e307a, en:e321]","[en:e927b, en:e570, en:e490, en:e422, en:e900,...",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459948754'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",,[],,[],[],[]
9,3560070923281,"[savon, mousant, carrefour, gel, dermatologiqu...","[en-e470-fr-added, en-e508-fr-added, en-e422-f...","[en:e330, en:e307a, en:e110]","[en:e470, en:e508, en:e422, en:e330, en:e1519,...",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1462099484'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]


In [115]:
df4.shape

(2619, 100)

In [116]:
df4.to_json('cleaned_data.json', orient='records')

In [117]:
### check to make sure the data saved as intended
cleaned_data = pd.read_json('cleaned_data.json')

In [118]:
cleaned_data.head()

,_id,_keywords,additives_debug_tags,additives_old_tags,additives_tags,allergens,allergens_hierarchy,allergens_tags,brands,brands_tags,...,sortkey,states,states_hierarchy,states_tags,stores,stores_tags,traces,traces_hierarchy,traces_tags,unknown_nutrients_tags
0,NaN,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,5.009785e+07,"[anti-trace, hygiene, alcool, bille, motionsen...",[],"[en:e173, en:e460]","[en:e173, en:e422, en:e428, en:e466, en:e211, ...",,[],[],"Rexona,Unilever","[rexona, unilever]",...,1487927561,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Franprix,[franprix],,[],[],[]
2,3.560070e+12,"[deodorant, musk, stick, carrefour, hygiene, a...","[en-e470-fr-added, en-e490-fr-added, en-e1520-...","[en:e1520, en:e110]","[en:e470, en:e490, en:e110]",,[],[],Carrefour,[carrefour],...,1491076400,"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour City,[carrefour-city],,[],[],[]
3,3.560071e+12,"[lait, le, tolerance, sou, corp, dermatologiqu...","[en-e422-fr-added, en-e211-fr-added, en-e524-f...","[en:e307a, en:e330]","[en:e422, en:e211, en:e524, en:e307a, en:e330]",,[],[],Carrefour,[carrefour],...,{u'$numberLong': u'1459971277'},"en:to-be-checked, en:complete, en:nutrition-fa...","[en:to-be-checked, en:complete, en:nutrition-f...","[en:to-be-checked, en:complete, en:nutrition-f...",Carrefour,[carrefour],,[],[],[]
4,3.560071e+12,"[karite, beurre, et, de, musquee, huile, carre...",[],[],[],,[],[],Carrefour,[carrefour],...,487925728,"en:to-be-completed, en:nutrition-facts-complet...","[en:to-be-completed, en:nutrition-facts-comple...","[en:to-be-completed, en:nutrition-facts-comple...",Carrefour,[carrefour],,[],[],[]


In [119]:
cleaned_data.shape

(2619, 100)